## 16. CNN Batch Norm

- 8번의 설명이랑 유사
- CNN에서는 **convolution 연산 후, 활성화 함수 적용 전에** BatchNorm을 적용
⇒ 정규화는 **각 배치의 같은 채널**을 기준으로 수행함

In [ ]:
import torch

# CPU/GPU 선택
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
import torch
import torch.nn as nn

input_tensor = torch.Tensor([[[[1, 2, 3, 4],
                             [5, 6, 7, 8],
                             [9, 10, 11, 12],
                             [13, 14, 15, 16],
                             ]]])

# Pooling layer 생성, 절반으로 줄이기
maxpool_layer = nn.MaxPool2d(kernel_size=2)
output_tensor = maxpool_layer(input_tensor)
print(f'Input Tensor shape: {input_tensor.shape}')
print(f'Output Tensor shape: {output_tensor.shape}')

Input Tensor shape: torch.Size([1, 1, 4, 4])
Output Tensor shape: torch.Size([1, 1, 2, 2])


In [ ]:
import torch
from torchvision import datasets, transforms

# 이미지 변환(전처리)
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR-10은 10개의 클래스에 걸쳐 총 60,000개의 32x32 컬러 이미지로 구성된 데이터셋
train_dataset = datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=4,
)

test_dataset = datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=4,
)

print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(test_dataset))



/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Train dataset size: 50000
Validation dataset size: 10000


In [ ]:
class ModernGAPCNN(nn.Module):
  def __init__(self, num_classes=10):
    super().__init__()

    self.feature_extractor = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=1), # 16 * 30 * 30
        # channel을 기준으로 batch 정규화
        nn.BatchNorm2d(16),
        nn.ReLU(inplace=True),
        # stride를 2로 설정하여 feature map 크기를 줄임(pooling 대체)
        nn.Conv2d(16, 32, kernel_size=5, stride=2, padding=1), # 32 * 15 * 15
        nn.BatchNorm2d(32),
        nn.ReLU(inplace=True),
        nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=1), # 64 * 7 * 17
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
    )

    # Global Average Pooling을 이용해 Channel별로 전체 값을 평균내어 1개의 값으로 축소
    self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1)) # 64 × 1 × 1
    self.classifier = nn.Linear(64, num_classes)

  def forward(self, x):
      x = self.feature_extractor(x)
      x = self.global_avg_pool(x)
      x = torch.flatten(x, 1)
      x = self.classifier(x)
      return x


In [13]:
import torch.optim as optim

net = ModernGAPCNN(10)
net.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(10):
  net.train()
  for batch_idx, (data, label) in enumerate(train_loader):
    data, label = data.to(device), label.to(device)
    optimizer.zero_grad()
    output = net(data)
    train_loss = loss_fn(output, label)
    train_loss.backward()
    optimizer.step()

  net.eval()
  val_loss = 0
  correct = 0
  with torch.no_grad():
    for data, label in test_loader:
      data, label = data.to(device), label.to(device)
      output = net(data)
      val_loss += loss_fn(output, label).item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(label.view_as(pred)).sum().item()

  val_loss /= len(test_loader.dataset)
  accuracy = 100. * correct / len(test_loader.dataset)

  print(f'Epoch {epoch+1}, Loss: {train_loss.item():.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')






Epoch 1, Loss: 1.9442, Val Loss: 0.3325, Accuracy: 52.87%
Epoch 2, Loss: 0.7816, Val Loss: 0.2853, Accuracy: 59.85%
Epoch 3, Loss: 0.4893, Val Loss: 0.2663, Accuracy: 62.05%
Epoch 4, Loss: 1.0107, Val Loss: 0.2509, Accuracy: 64.83%
Epoch 5, Loss: 1.1344, Val Loss: 0.2421, Accuracy: 66.15%
Epoch 6, Loss: 2.3736, Val Loss: 0.2262, Accuracy: 68.12%
Epoch 7, Loss: 0.3943, Val Loss: 0.2182, Accuracy: 69.69%
Epoch 8, Loss: 1.3765, Val Loss: 0.2155, Accuracy: 70.19%
Epoch 9, Loss: 1.3116, Val Loss: 0.2207, Accuracy: 69.06%
Epoch 10, Loss: 1.3179, Val Loss: 0.2042, Accuracy: 71.96%
